In [2]:
import subprocess
import sys
import os

# Define a function to install packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages to install
packages = [
    "pandas",
    "numpy",
    "scikit-learn",
    "pickle-mixin",
    "pyarrow"
]

# Install each package
for package in packages:
    try:
        __import__(package)
    except ImportError:
        install(package)

#assign correct directory
name = sys.platform

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVC
import random as rm
from sklearn.metrics import classification_report, f1_score
import pickle as pk
from sklearn.preprocessing import PolynomialFeatures
from itertools import combinations
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Lasso
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import mahalanobis

In [14]:
import pandas as pd

# 1. Load the Stata dataset
df = pd.read_stata("CMF/CMF_1860.dta")

# 2. Convert "materials_value*" and "production_values*" columns to numeric (force conversion)
# Identify columns whose names start with either pattern and convert them (errors='coerce' converts non-numeric values to NaN)
for col in df.columns:
    if col.startswith("materials_value") or col.startswith("production_values"):
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Generate aggregated features for "materials_value" and "production_values"
# For each base variable, sum all columns whose names start with that variable.
for base in ['materials_value', 'production_values']:
    matching_cols = [col for col in df.columns if col.startswith(base)]
    # Sum across the matching columns. The skipna=True means only non-missing numbers are summed.
    df[base] = df[matching_cols].sum(axis=1, skipna=True)
    # In case all values were missing, replace NaN with 0.
    df[base] = df[base].fillna(0)

# 4. Rename the aggregated 'production_values' column to 'output'
df.rename(columns={'production_values': 'output'}, inplace=True)

# 5. Keep only specific columns.
#    The Stata "keep" command retains:
#    file_name, firm_number, firm_name, output, capital, materials_value, hands_female,
#    hands_male, avg_wage_female, avg_wage_male, all variables starting with "ind_",
#    materials_value1-5, and production_values1-5.
# First build a list of columns to keep:
base_keep = [
    'file_name', 'firm_number', 'firm_name', 'output', 'capital',
    'materials_value', 'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male'
]
# Add any columns starting with "ind_"
ind_cols = [col for col in df.columns if col.startswith("ind_")]
# Explicitly add materials_value1 through materials_value5 if they exist
mv_cols = [f"materials_value{i}" for i in range(1, 6) if f"materials_value{i}" in df.columns]
# Explicitly add production_values1 through production_values5 if they exist
# (These will later be renamed)
pv_cols = [f"production_values{i}" for i in range(1, 6) if f"production_values{i}" in df.columns]

# Retain only these columns (if they exist in the dataframe)
cols_to_keep = base_keep + ind_cols + mv_cols + pv_cols
df = df[cols_to_keep]

# 6. For specified numeric columns, ensure they are numeric and replace missing values with 0.
#    The variables are: output, capital, materials_value, hands_female, hands_male,
#    avg_wage_female, and avg_wage_male.
for col in ['output', 'capital', 'materials_value', 'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# 7. Rename production values variables
#    In Stata, the loop renames each variable starting with "production_values" (other than the aggregated one)
#    to output1, output2, etc.
# Find all columns that start with "production_values". (These are the individual ones kept earlier.)
pv_individual = [col for col in df.columns if col.startswith("production_values")]
for i, col in enumerate(pv_individual, start=1):
    df.rename(columns={col: f"output{i}"}, inplace=True)

# 8. Reorder columns so that a specific set of variables come first.
#    The desired order (moved to front) is:
#    file_name, firm_number, firm_name, capital, materials_value, hands_female,
#    hands_male, avg_wage_female, avg_wage_male, output.
#    Other columns will follow in their original order.
front_cols = ['file_name', 'firm_number', 'firm_name', 'capital', 'materials_value',
              'hands_female', 'hands_male', 'avg_wage_female', 'avg_wage_male', 'output']
remaining_cols = [col for col in df.columns if col not in front_cols]
df = df[front_cols + remaining_cols]

# 9. Save the resulting dataframe to a Stata .dta file.
output_file = r"/Users/paulwang/Downloads/create_corrections_files/CMF_to_predict/1860_to_predict.dta"
df.to_stata(output_file, write_index=False)

print("Data processing complete. Output saved to:", output_file)

/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_39891/185487611.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[base] = df[matching_cols].sum(axis=1, skipna=True)
/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_39891/185487611.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[base] = df[matching_cols].sum(axis=1, skipna=True)


Data processing complete. Output saved to: /Users/paulwang/Downloads/create_corrections_files/CMF_to_predict/CMF_1860_to_predict.dta


In [4]:
def ratio(df, var1, var2):
    df[f'{var1}&{var2}'] = df[var1]/df[var2]


In [5]:
# Read data
input_1850 = pd.read_stata("1850_to_predict.dta")

#create specific ratios

ratio(input_1850, 'materials_value', 'output')
ratio(input_1850, 'materials_value', 'capital')
ratio(input_1850, 'capital', 'output')
ratio(input_1850, 'capital', 'materials_value')
ratio(input_1850, 'output', 'materials_value')
ratio(input_1850, 'output', 'capital')
ratio(input_1850, 'hands_male', 'avg_wage_male')
ratio(input_1850, 'hands_female', 'avg_wage_female')
ratio(input_1850, 'avg_wage_male', 'output')
ratio(input_1850, 'avg_wage_female', 'output')

var_list = ["capital", "materials_value", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output", 'materials_value&output', 'materials_value&capital', 'capital&output', 'capital&materials_value', 'output&materials_value', 'output&capital','hands_male&avg_wage_male', 'hands_female&avg_wage_female', 'avg_wage_male&output', 'avg_wage_female&output' ]

input_1850.replace({np.inf: 0, -np.inf: 0, np.nan: 0}, inplace=True)


In [6]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis

industry_list = input_1850["ind_broadest"].unique().tolist()
final_df = []

for industry in industry_list:
    input_df = input_1850[input_1850["ind_broadest"] == industry].copy()  # Ensure a copy

    if input_df.empty:
        continue  # Skip if there's no data for this industry

    # Compute mean and covariance matrix
    mean_vec = np.mean(input_df[var_list], axis=0)
    cov_matrix = np.cov(input_df[var_list], rowvar=False)

    # Check if covariance matrix is singular
    if np.linalg.det(cov_matrix) == 0:
        print(f"Skipping {industry}: Covariance matrix is singular")
        continue

    inv_cov_matrix = np.linalg.inv(cov_matrix)

    # Compute Mahalanobis distance for each firm
    distances = np.array([mahalanobis(row, mean_vec, inv_cov_matrix) for row in input_df[var_list].to_numpy()])
    
    # Compute contributions
    contributions = np.array([(row - mean_vec) * np.dot(inv_cov_matrix, (row - mean_vec)) for row in input_df[var_list].to_numpy()])
    contributions_df = pd.DataFrame(contributions, columns=[col + "_con" for col in var_list])

    # Identify the most anomalous variable for each row
    input_df['most_anomalous_var'] = contributions_df.idxmax(axis=1)
    input_df['mahalanobis'] = distances

    # Combine input_df with contributions_df (ensuring correct alignment)
    input_df = pd.concat([input_df.reset_index(drop=True), contributions_df.reset_index(drop=True)], axis=1)

    final_df.append(input_df)  # Append a COPY

# Combine all industry DataFrames into one
final_df = pd.concat(final_df, ignore_index=True)

In [8]:
final_df['dist_squared'] = final_df['mahalanobis'] ** 2
for var in var_list:
    final_df[var + "_divided"] = final_df[var+"_con"]/final_df['dist_squared']
final_df['total'] = 0
for var in var_list:
    final_df['total'] = final_df['total'] + final_df[var + "_divided"]
final_df

,file_name,firm_number,firm_name,capital,materials_value,hands_female,hands_male,avg_wage_female,avg_wage_male,output,...,materials_value&capital_divided,capital&output_divided,capital&materials_value_divided,output&materials_value_divided,output&capital_divided,hands_male&avg_wage_male_divided,hands_female&avg_wage_female_divided,avg_wage_male&output_divided,avg_wage_female&output_divided,total
0,AL_5__00029_r_L.jpg,1,Chickasan Mill Co,7000.0,2000.0,0.0,12.0,0.0,252.0,15000.0,...,-0.005190,0.010934,0.000081,-0.001303,-0.000433,2.139137e-04,0.003695,0.032064,-0.000101,1.0
1,AL_5__00029_r_L.jpg,3,James Thogmorton,7000.0,2800.0,0.0,10.0,0.0,150.0,8000.0,...,0.006735,0.014848,0.000315,0.000519,0.002135,-2.308508e-07,0.019228,0.025239,-0.001441,1.0
2,AL_5__00029_r_L.jpg,20,Justman Williams,2000.0,1000.0,0.0,2.0,0.0,24.0,1800.0,...,0.036506,0.005276,0.006371,-0.004030,0.023789,3.703288e-02,0.038786,0.439638,-0.003770,1.0
3,AL_5__00029_r_L.jpg,22,John F Pride,3000.0,1000.0,0.0,3.0,0.0,60.0,3000.0,...,0.094847,-0.000854,-0.000454,-0.001290,0.011824,8.461813e-02,0.079545,0.135612,-0.001981,1.0
4,AL_5_autauga_00003_r_L.jpg,1,John Steele,0.0,0.0,0.0,2.0,0.0,30.0,1000.0,...,0.039524,0.210420,0.017073,-0.005072,0.056121,2.261885e-08,-0.001092,0.018531,0.004205,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125056,WI_5_waukesha_00103_l_L.jpg,5,Cutter & Dakin,2000.0,0.0,0.0,8.0,0.0,124.0,2000.0,...,0.037935,-0.002800,0.011049,-0.005763,0.241323,4.208800e-02,0.027212,0.004327,0.008479,1.0
125057,WI_5_waukesha_00103_l_L.jpg,10,Hekirden & Cook,2000.0,0.0,0.0,4.0,0.0,104.0,1600.0,...,0.033452,-0.002223,0.014679,-0.004766,0.278688,1.404878e-02,0.015424,0.004932,0.008419,1.0
125058,WI_5_waukesha_00103_l_L.jpg,11,Silas Hermington,150.0,0.0,0.0,2.0,0.0,50.0,525.0,...,0.040005,-0.001357,0.005447,-0.003978,0.159259,9.043035e-03,0.016083,0.001782,0.008399,1.0
125059,dc_5_washington city_00062_r_L.jpg,9,Washingtons National Monument Society,96806.0,0.0,0.0,50.0,0.0,1922.0,58.0,...,-0.000015,0.991125,0.001336,-0.000310,-0.000032,-4.486156e-05,0.000011,-0.001013,0.000003,1.0


In [9]:
#output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output"]

for var in var_list:
    batch = []
    for ind in industry_list:
        ind_specific = final_df[final_df["ind_broadest"] == ind].copy()  # Ensure a copy
        #ORIGINAL CUTOFF : .0001
        quantile_low = ind_specific[var + '_divided'].quantile(0.0001)
        quantile_high = ind_specific[var + '_divided'].quantile(1 - 0.0001)

        # Filter the lowest and highest 0.01% values
        lowest_values = ind_specific[ind_specific[var + '_divided'] <= quantile_low]
        highest_values = ind_specific[ind_specific[var + '_divided'] >= quantile_high]

        # Combine results
        result = pd.concat([lowest_values, highest_values])

        batch.append(result)

    batch = pd.concat(batch, ignore_index=True)

    if "materials_value&output" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        variable_check['correct_materials_value_1'] = None
        variable_check['correct_materials_value_2'] = None
        variable_check['correct_materials_value_3'] = None
        variable_check['correct_materials_value_4'] = None
        variable_check['correct_materials_value_5'] = None
        variable_check['correct_output_1'] = None
        variable_check['correct_output_2'] = None
        variable_check['correct_output_3'] = None
        variable_check['correct_output_4'] = None
        variable_check['correct_output_5'] = None
        variable_check.to_csv(f"{var}_check_1850.csv", index=False)
    elif "output&materials_value" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        variable_check['correct_output_1'] = None
        variable_check['correct_output_2'] = None
        variable_check['correct_output_3'] = None
        variable_check['correct_output_4'] = None
        variable_check['correct_output_5'] = None
        variable_check['correct_materials_value_1'] = None
        variable_check['correct_materials_value_2'] = None
        variable_check['correct_materials_value_3'] = None
        variable_check['correct_materials_value_4'] = None
        variable_check['correct_materials_value_5'] = None
        variable_check.to_csv(f"{var}_check_1850.csv", index=False)

    elif "materials_value" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value1", "materials_value2", "materials_value3", "materials_value4", "materials_value5", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        parts = var.split("&")
        if len(parts) == 2:
            left, right=parts
            if "materials_value" in left:
                variable_check['correct_materials_value_1'] = None
                variable_check['correct_materials_value_2'] = None
                variable_check['correct_materials_value_3'] = None
                variable_check['correct_materials_value_4'] = None
                variable_check['correct_materials_value_5'] = None
                variable_check[f'correct_{right}'] = None
            else:
                variable_check[f'correct_{left}'] = None
                variable_check['correct_materials_value_1'] = None
                variable_check['correct_materials_value_2'] = None
                variable_check['correct_materials_value_3'] = None
                variable_check['correct_materials_value_4'] = None
                variable_check['correct_materials_value_5'] = None
        else:
            variable_check['correct_materials_value_1'] = None
            variable_check['correct_materials_value_2'] = None
            variable_check['correct_materials_value_3'] = None
            variable_check['correct_materials_value_4'] = None
            variable_check['correct_materials_value_5'] = None
        variable_check.to_csv(f"{var}_check_1850.csv", index=False)

    elif "output" in var:
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output1", "output2", "output3", "output4", "output5"]
        variable_check = batch[output_var_list]
        variable_check['transcription_error'] =  None
        parts = var.split("&")
        if len(parts) == 2:
            left, right=parts
            if "output" in left:
                variable_check['correct_output_value_1'] = None
                variable_check['correct_output_value_2']= None
                variable_check['correct_output_value_3'] = None
                variable_check['correct_output_value_4'] = None
                variable_check['correct_output_value_5'] = None
                variable_check[f'correct_{right}'] = None
            else:
                variable_check[f'correct_{left}'] = None
                variable_check['correct_output_value_1'] = None
                variable_check['correct_output_value_2'] = None
                variable_check['correct_output_value_3'] = None
                variable_check['correct_output_value_4'] = None
                variable_check['correct_output_value_5'] = None
        else:
            variable_check['correct_output_value_1'] = None
            variable_check['correct_output_value_2'] = None
            variable_check['correct_output_value_3'] = None
            variable_check['correct_output_value_4'] = None
            variable_check['correct_output_value_5'] = None
        variable_check.to_csv(f"{var}_check_1850.csv", index=False)

    else:
        parts = var.split("&")
        output_var_list = ["file_name", "firm_number", "firm_name", "capital", "materials_value", "hands_female", "hands_male", "avg_wage_female", "avg_wage_male", "output"]
        variable_check = batch[output_var_list]
        if len(parts) == 2:
            left, right=parts
            variable_check['transcription_error'] =  None
            variable_check[f'correct_{left}'] = None
            variable_check[f'correct_{right}'] = None
        else:
            variable_check['transcription_error'] =  None
            variable_check[f'correct_{var}'] = None
        variable_check.to_csv(f"{var}_check_1850.csv", index=False)



/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_39891/568361458.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variable_check['transcription_error'] =  None
/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_39891/568361458.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variable_check[f'correct_{var}'] = None
/var/folders/9v/swhhkcgn3h543bcvl6qz76lc0000gn/T/ipykernel_39891/568361458.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF